In [ ]:
# load required libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm

In [ ]:
# install & load torchsummary to preint summary of model
!pip install torchsummary
from torchsummary import summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# construct cnn class
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size = 3)  # apply 16 channel convolution with kernel size 3*3 on image
        self.conv2 = nn.Conv2d(16, 16, kernel_size = 3)  # apply 16 channel convolution with kernel size 3*3 on image
        self.conv3 = nn.Conv2d(16, 16, kernel_size = 3)  # apply 16 channel convolution with kernel size 3*3 on image
        self.conv4 = nn.Conv2d(16, 16, kernel_size = 3)  # apply 16 channel convolution with kernel size 3*3 on image
        self.conv5 = nn.Conv2d(16, 64, kernel_size = 3)  # apply 64 channel convolution with kernel size 3*3 on image
        self.batch_norm1 = nn.BatchNorm2d(16)  # batch normalization on convolution layer 1
        self.batch_norm2 = nn.BatchNorm2d(16)  # batch normalization on convolution layer 2
        self.batch_norm3 = nn.BatchNorm2d(16)  # batch normalization on convolution layer 3
        self.batch_norm4 = nn.BatchNorm2d(16)  # batch normalization on convolution layer 4
        self.batch_norm5 = nn.BatchNorm2d(64)  # batch normalization on convolution layer 5
        self.batch_norm6 = nn.BatchNorm1d(32)  # batch normalization on dense layer after gap and before output layer
        self.dropout1 = nn.Dropout(p = 0.2)  # dropout on convolution layer 2
        self.dropout2 = nn.Dropout(p = 0.2)  # dropout on convolution layer 3
        self.dropout3 = nn.Dropout(p = 0.2)  # dropout on convolution layer 4
        self.dropout4 = nn.Dropout(p = 0.1)  # dropout after global average pooling
        self.dropout5 = nn.Dropout(p = 0.1)  # dropout on dense layer
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(64, 32)  # dense layer after global average pooling layer
        self.fc2 = nn.Linear(32, 10)  # output layer

    def forward(self, img):
        x = F.relu(self.conv1(img))  # input = 1 * 28 * 28, output = 8 * 26 * 26
        x = self.batch_norm1(x)  # batch normalization

        x = F.relu(self.conv2(x))  # input = 8 * 26 * 26, output = 16 * 24 * 24
        x = self.batch_norm2(x)  # batch normalization
        x = F.max_pool2d(x, 2)  # input = 16 * 24 * 24, output = 16 * 12 * 12
        x = self.dropout1(x)  # drop out

        x = F.relu(self.conv3(x))  # input = 16 * 12 * 12, output = 16 * 10 * 10
        x = self.batch_norm3(x)  # batch normalization
        x = self.dropout2(x)  # drop out

        x = F.relu(self.conv4(x))  # input = 16 * 10 * 10, output = 16 * 8 * 8
        x = self.batch_norm4(x)  # batch normalization
        x = self.dropout3(x)  # drop out

        x = F.relu(self.conv5(x))  # input = 16 * 8 * 8, output = 64 * 6 * 6
        x = self.batch_norm5(x)  # batch normalization
       
        x = self.gap(x)  # global average pooling - input = 64 * 6 * 6, output = 64 * 1 * 1
        x = x.view(-1, 64)  # flatten GAP embedding
        x = self.dropout4(x)  # drop out

        x = F.relu(self.fc1(x))
        x = self.batch_norm6(x)  # batch normalization
        x = self.dropout5(x)  # drop out

        img_out = F.log_softmax(self.fc2(x), dim = 1)  # output layer to output probabilities for label

        return img_out

In [ ]:
# find if device is gpu
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# move model to gpu
model = Net().to(device)

#print model summary
summary(model, input_size = (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
            Conv2d-3           [-1, 16, 24, 24]           2,320
       BatchNorm2d-4           [-1, 16, 24, 24]              32
           Dropout-5           [-1, 16, 12, 12]               0
            Conv2d-6           [-1, 16, 10, 10]           2,320
       BatchNorm2d-7           [-1, 16, 10, 10]              32
           Dropout-8           [-1, 16, 10, 10]               0
            Conv2d-9             [-1, 16, 8, 8]           2,320
      BatchNorm2d-10             [-1, 16, 8, 8]              32
          Dropout-11             [-1, 16, 8, 8]               0
           Conv2d-12             [-1, 64, 6, 6]           9,280
      BatchNorm2d-13             [-1, 64, 6, 6]             128
AdaptiveAvgPool2d-14             [-1, 6

In [ ]:
# random seed for replication
torch.manual_seed(1)
# batch size
batch_size = 32

# train data loader
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

# test data loader
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [ ]:
# model training function
def train(model, device, train_loader, optimizer, epoch):
    model.train()  # initiate model training state
    pbar = tqdm(train_loader)  # progress bar
    for batch_idx, (data, target) in enumerate(pbar):  # iterate over batches in data
        data, target = data.to(device), target.to(device)  # move data to gpu
        optimizer.zero_grad()  # clear out gradients
        output = model(data)  # predict output from model
        loss = F.nll_loss(output, target)  # calculate negative log likelihood loss
        loss.backward()  # backward propogation of loss
        optimizer.step()  # update gradients
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')  # update progress bar

# model evaluation function
def test(model, device, test_loader):
    model.eval()  # initiate model evaluation state
    test_loss = 0  # initailse test loss as 0
    correct = 0  # number of correct samples
    with torch.no_grad():  # specify that no gradients will be used
        for data, target in test_loader:  # iterate over batches in data
            data, target = data.to(device), target.to(device)  # move data to gpu
            output = model(data)  # predict output from model
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)  # average loss

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
# train & evaluate model
epochs = 20  # number of epochs to train modle
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.9)  # optimizer

for epoch in range(1, epochs):
    train(model, device, train_loader, optimizer, epoch)  # training
    test(model, device, test_loader)  # evaluation

loss=0.017135070636868477 batch_id=1874: 100%|██████████| 1875/1875 [00:31<00:00, 59.41it/s]



Test set: Average loss: 0.0384, Accuracy: 9868/10000 (98.68%)



loss=0.023957887664437294 batch_id=1874: 100%|██████████| 1875/1875 [00:28<00:00, 65.60it/s]



Test set: Average loss: 0.0338, Accuracy: 9885/10000 (98.85%)



loss=0.02425348572432995 batch_id=1874: 100%|██████████| 1875/1875 [00:29<00:00, 64.27it/s]


...


Test set: Average loss: 0.0187, Accuracy: 9942/10000 (99.42%)



loss=0.17401878535747528 batch_id=1874: 100%|██████████| 1875/1875 [00:29<00:00, 63.58it/s]



Test set: Average loss: 0.0158, Accuracy: 9951/10000 (99.51%)



loss=0.07753217220306396 batch_id=1874: 100%|██████████| 1875/1875 [00:28<00:00, 64.85it/s]



Test set: Average loss: 0.0169, Accuracy: 9950/10000 (99.50%)



In [ ]:
# train performance
test(model, device, train_loader)  # evaluation


Test set: Average loss: 0.0120, Accuracy: 59782/60000 (99.64%)



In [ ]:
# test performance
test(model, device, test_loader)  # evaluation


Test set: Average loss: 0.0169, Accuracy: 9950/10000 (99.50%)

